<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab


---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [1]:
#!pip install twitter python-twitter

## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [1]:
import pandas as pd
import numpy as np

In [2]:
import twitter, re, datetime, pandas as pd
from local_settings import twitter_keys

"""
twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAIr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61lTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-dYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTboz9iIyp8QxX6PqRApdi49baLqBgzo'
}
"""
api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [4]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'KmN03M1X1pImZ43sqdIu4yfnE',
        'consumer_secret':     'ePlIrIX5VXbZnO7DBu1RbFlw5lOai9dQr9n5TZb6vxnIdrr5Fz',
        'access_token_key':    '185036086-Q7K5IjuSoQZJwSIqD0wyHf6t62iPKatmfaPkriAM',
        'access_token_secret': 'cYpQz3xWHQbLplOj8iSeiNSOmMcsTOXmWcKMrJ9buLj5d'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [5]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [6]:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [7]:
print sanders[0]

{'handle': u'Bernie Sanders', 'mined_at': datetime.datetime(2017, 4, 27, 16, 4, 0, 211822), 'created_at': u'Wed Apr 26 21:08:43 +0000 2017', 'tweet_id': 857340689629237248, 'text': u'RT @keithellison: On the same day Democrats were rallying around a $15 minimum wage, the @GOP unveiled a massive tax cut for corporations a\u2026', 'retweet_count': 1991}


In [8]:
print donald[0]

{'handle': u'Donald J. Trump', 'mined_at': datetime.datetime(2017, 4, 27, 16, 4, 2, 159150), 'created_at': u'Thu Apr 27 14:39:25 +0000 2017', 'tweet_id': 857605104417026049, 'text': u'As families prepare for summer vacations in our National Parks - Democrats threaten to close them and shut down the government. Terrible!', 'retweet_count': 2701}


### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [9]:
pd.DataFrame(sanders).head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Apr 26 21:08:43 +0000 2017,Bernie Sanders,2017-04-27 16:04:00.211822,1991,RT @keithellison: On the same day Democrats we...,857340689629237248
1,Wed Apr 26 20:57:23 +0000 2017,Bernie Sanders,2017-04-27 16:04:00.211833,2473,We will not rest until every worker in America...,857337838563725312
2,Wed Apr 26 17:12:16 +0000 2017,Bernie Sanders,2017-04-27 16:04:00.480417,339,RT @danielmarans: Bernie Sanders and Democrats...,857281186217041921
3,Mon Apr 24 21:52:25 +0000 2017,Bernie Sanders,2017-04-27 16:04:00.480430,186,RT @NationalAction: Bernie Sanders will be add...,856626910826287106
4,Mon Apr 24 20:24:47 +0000 2017,Bernie Sanders,2017-04-27 16:04:00.801146,114,RT @LateNightSeth: We’re back with a great wee...,856604855867240448


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [10]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = TweetMiner(twitter_keys, api, result_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [11]:
trump_df = pd.DataFrame(trump_tweets)
print trump_df.shape

(999, 6)


In [12]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [13]:
bernie_df = pd.DataFrame(bernie_tweets) 
print bernie_df.shape

(1000, 6)


In [14]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(1999, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4), stop_words="english")

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'fake news', 39),
 (u'make america', 33),
 (u'america great', 29),
 (u'watch https', 28),
 (u'make america great', 27),
 (u'united states', 23),
 (u'failing nytimes', 18),
 (u'white house', 17),
 (u'tickets https', 15),
 (u'prime minister', 14),
 (u'maga https', 14),
 (u'news media', 12),
 (u'fake news media', 11),
 (u'looking forward', 10),
 (u'north carolina', 10),
 (u'icymi watch', 10),
 (u'donald trump', 10),
 (u'president elect', 10),
 (u'hillary clinton', 10),
 (u'president obama', 9)]

### Look at the ngrams for Bernie Sanders

In [18]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4), stop_words="english")

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'health care', 76),
 (u'donald trump', 64),
 (u'bernie sanders', 47),
 (u'climate change', 28),
 (u'wall street', 25),
 (u'working families', 24),
 (u'democratic party', 23),
 (u'american people', 22),
 (u'tax breaks', 19),
 (u'working people', 18),
 (u'https rt', 16),
 (u'young people', 14),
 (u'hillary clinton', 14),
 (u'mr trump', 14),
 (u'millions people', 13),
 (u'minimum wage', 13),
 (u'health insurance', 12),
 (u'drug companies', 11),
 (u'middle class', 11),
 (u'fair share', 11)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [20]:
# BONUS
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [21]:
print tweet_text[0:3]

[ u'As families prepare for summer vacations in our National Parks - Democrats threaten to close them and shut down the government. Terrible!'
 u'Democrats used to support border security \u2014 now they want illegals to pour through our borders.'
 u'Democrats jeopardizing the safety of our troops to bail out their donors from insurance companies. It is time to put #AmericaFirst\U0001f1fa\U0001f1f8']


In [22]:
print clean_text[0:3]

[u'as families prepare for summer vacations in our national parks democrats threaten to close them and shut down the government terrible', u'democrats used to support border security \u2014 now they want illegals to pour through our borders', u'democrats jeopardizing the safety of our troops to bail out their donors from insurance companies it is time to put americafirst\ufffd\ufffd\ufffd\ufffd']


In [25]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['handle'].map(lambda x: 1 if x == 'Donald J. Trump' else 0).values
print np.mean(y)

0.499749874937


In [26]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
print X.shape

(1999, 2000)


In [27]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

import numpy as np
print accuracies
print np.mean(accuracies)

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)


[ 0.87        0.92        0.925       0.935       0.91        0.925       0.875
  0.885       0.85        0.84422111]
0.893922110553


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
# Very good accuracy considering the baseline is 50%

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [29]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

Xtest = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(Xtest)


array([[ 0.81518984,  0.18481016],
       [ 0.28077741,  0.71922259]])

In [43]:
# The 1st column is probability of being Bernie, and 2nd Trump. The classifier is getting it right.

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [30]:
# We deviate from trump / sanders using student tweets here to illustrate the NLP pipeine with twitter data

twitter_handles = ["dril", "LaziestCanine",'ch000ch']
tweets = {}

for twitter_handle in twitter_handles:
    print "Mining tweets for: ", twitter_handle
    miner = TweetMiner(twitter_keys, api, result_limit=500)
    tweets[twitter_handle] = miner.mine_user_tweets(user=twitter_handle, max_pages=10)


Mining tweets for:  dril
Mining tweets for:  LaziestCanine
Mining tweets for:  ch000ch


In [31]:
multi = pd.DataFrame(tweets['dril'])
multi = multi.append(pd.DataFrame(tweets['LaziestCanine']))
multi = multi.append(pd.DataFrame(tweets['ch000ch']))

print multi.shape


(5993, 6)


In [32]:
multi.handle.value_counts()

Lazy dog    1999
wint        1998
chuuch      1996
Name: handle, dtype: int64

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [33]:
tweet_text = multi['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

y = multi['handle'].map(lambda x: 0 if x == 'wint' else 1 if x == 'Lazy dog' else 2).values

In [34]:
tfv = TfidfVectorizer(ngram_range=(1,3), max_features=2500)
X = tfv.fit_transform(clean_text)

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

rf = RandomForestClassifier(n_estimators=250, verbose=1)
knn = KNeighborsClassifier(n_neighbors=7)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    6.9s finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [37]:
# Random forest score:
print 'RF:', rf.score(X_test, y_test)
print 'KNN:', knn.score(X_test, y_test)

RF:

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.4s finished


 0.704115684093
KNN: 0.414349276974


In [38]:
# Baseline score:
multi.handle.value_counts()/multi.shape[0]

Lazy dog    0.333556
wint        0.333389
chuuch      0.333055
Name: handle, dtype: float64

In [39]:
rf_yhat = knn.predict(X_test)

### 3. Make a confusion matrix and classification report.

In [40]:
from sklearn.metrics import classification_report, confusion_matrix

print classification_report(y_test, rf_yhat)


             precision    recall  f1-score   support

          0       0.51      0.22      0.31       636
          1       0.38      0.72      0.49       584
          2       0.45      0.31      0.37       578

avg / total       0.45      0.41      0.39      1798



In [41]:
# Confusion Matrix
print confusion_matrix(y_test, rf_yhat)

[[143 362 131]
 [ 76 421  87]
 [ 62 335 181]]


### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [42]:
rf.fit(X, y)
pp = rf.predict_proba(X)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   10.6s finished
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.9s finished


In [43]:
pp[0:5]

array([[ 0.672     ,  0.248     ,  0.08      ],
       [ 0.49742617,  0.4643658 ,  0.03820803],
       [ 0.936     ,  0.008     ,  0.056     ],
       [ 0.828     ,  0.084     ,  0.088     ],
       [ 0.824     ,  0.032     ,  0.144     ]])

In [44]:
pp = pd.DataFrame(pp, columns=['dril_pp', 'laziestcanine_pp', 'ch000ch_pp'])

In [45]:
print multi.shape, pp.shape

(5993, 6) (5993, 3)


In [46]:
tweets_pp = pd.concat([multi.reset_index(), pp.reset_index()], axis=1)
tweets_pp.head(2)

,index,created_at,handle,mined_at,retweet_count,text,tweet_id,index,dril_pp,laziestcanine_pp,ch000ch_pp
0,0,Mon Apr 24 01:58:26 +0000 2017,wint,2017-04-27 16:09:45.137602,779,wish that was me https://t.co/p3SQmVBZ6R,856326435144962048,0,0.672000,0.248000,0.080000
1,1,Mon Apr 24 01:17:19 +0000 2017,wint,2017-04-27 16:09:45.137615,631,Traitors https://t.co/zgFjQZgjaH,856316087893450752,1,0.497426,0.464366,0.038208


In [47]:
print 'Most dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=False).text.values[0]
print 'Least dril:', tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=True).text.values[0]

Most dril: @Insubordination shut the fuck up
Least dril: @antonwheel thanks


In [48]:
print 'Most LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=False).text.values[0]
print 'Least LaziestCanine:', tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=True).text.values[0]

Most LaziestCanine: RT @LaziestCanine: "the bee movie isn't an American Classic" https://t.co/Q5ffYebaFj
Least LaziestCanine: @J_Millie42 lmao


In [49]:
print 'Most chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=False).text.values[0]
print 'Least chuuch:', tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=True).text.values[0]

Most chuuch: @BuckyIsotope  https://t.co/shlEMOluW6
Least chuuch: https://t.co/8q4CWszajS
